In [1]:
from google.colab import files
uploaded = files.upload()

Saving DirtyRetailSales - DirtyRetailSales.csv to DirtyRetailSales - DirtyRetailSales.csv


In [15]:
import pandas as pd
import sqlite3
import glob

# Auto-pick your CSV (edit the pattern if needed)
candidates = [p for p in glob.glob("*.csv") if "DirtyRetailSales" in p or "Retail" in p]
CSV_FILE = "DirtyRetailSales - DirtyRetailSales.csv"
print("Using:", CSV_FILE)

# Load raw CSV into pandas
df = pd.read_csv(CSV_FILE, low_memory=False)

# Convert pandas NaN -> real SQL NULLs
df = df.where(pd.notnull(df), None)

# Connect to SQLite DB (file saved in the Colab working dir)
conn = sqlite3.connect("cleaning_project.db")

# Write raw -> SQL
df.to_sql("raw_data", conn, if_exists="replace", index=False)
print("Rows in raw_data:", pd.read_sql_query("SELECT COUNT(*) c FROM raw_data;", conn)["c"][0])

Using: DirtyRetailSales - DirtyRetailSales.csv
Rows in raw_data: 12575


In [16]:
def q(sql: str):
    """Run a SELECT and return DataFrame."""
    return pd.read_sql_query(sql, conn)

def x(sql: str):
    """Run DDL/DML (no result set)."""
    conn.executescript(sql)

In [17]:
x("""
DROP TABLE IF EXISTS stage;

CREATE TABLE stage AS
SELECT
  TRIM("Transaction ID")                         AS transaction_id,
  TRIM("Customer ID")                            AS customer_id,
  TRIM(Category)                                 AS category,
  TRIM(Item)                                     AS item,
  CAST(REPLACE(TRIM("Price Per Unit"), '$','') AS REAL) AS price_per_unit,
  CAST(TRIM(Quantity) AS INTEGER)                AS quantity,
  CAST(REPLACE(TRIM("Total Spent"), '$','')  AS REAL)   AS total_spent,
  LOWER(TRIM("Payment Method"))                  AS payment_method,
  LOWER(REPLACE(TRIM(Location), '-', ' '))       AS location,
  /* keep as TEXT for now; normalize later; julianday(NULL)=NULL for bad dates */
  CASE
    WHEN julianday(TRIM("Transaction Date")) IS NOT NULL THEN DATE(TRIM("Transaction Date"))
    ELSE NULL
  END                                            AS transaction_date,
  CASE
    WHEN UPPER(TRIM("Discount Applied")) IN ('TRUE','T','YES','Y','1')  THEN 1
    WHEN UPPER(TRIM("Discount Applied")) IN ('FALSE','F','NO','N','0') THEN 0
    ELSE NULL
  END                                            AS discount_applied
FROM raw_data;
""")

q("SELECT * FROM stage LIMIT 5;")

,transaction_id,customer_id,category,item,price_per_unit,quantity,total_spent,payment_method,location,transaction_date,discount_applied
0,TXN_6867343,CUST_09,Patisserie,Item_10_PAT,18.5,10,185.0,digital wallet,online,2024-04-08,1.0
1,TXN_3731986,CUST_22,Milk Products,Item_17_MILK,29.0,9,261.0,digital wallet,online,2023-07-23,1.0
2,TXN_9303719,CUST_02,Butchers,Item_12_BUT,21.5,2,43.0,credit card,online,2022-10-05,0.0
3,TXN_9458126,CUST_06,Beverages,Item_16_BEV,27.5,9,247.5,credit card,online,2022-05-07,NaN
4,TXN_4575373,CUST_05,Food,Item_6_FOOD,12.5,7,87.5,digital wallet,online,2022-10-02,0.0


In [18]:
x("""
DROP TABLE IF EXISTS cleaned_dedup;

CREATE TABLE cleaned_dedup AS
WITH ranked AS (
  SELECT *,
         ROW_NUMBER() OVER (
           PARTITION BY transaction_id, customer_id, category, item,
                        price_per_unit, quantity, total_spent,
                        payment_method, location, transaction_date, discount_applied
           ORDER BY rowid
         ) AS rn
  FROM stage
)
SELECT * FROM ranked WHERE rn = 1;
""")

q("SELECT COUNT(*) AS rows_cleaned_dedup FROM cleaned_dedup;")

,rows_cleaned_dedup
0,12575


In [19]:
x("""
DROP TABLE IF EXISTS cleaned_final;

CREATE TABLE cleaned_final AS
SELECT
  COALESCE(transaction_id, 'unknown')                    AS transaction_id,
  COALESCE(customer_id,  'unknown')                      AS customer_id,
  COALESCE(category,     'unknown')                      AS category,
  COALESCE(item,         'unknown')                      AS item,
  COALESCE(price_per_unit, 0.0)                          AS price_per_unit,
  COALESCE(quantity, 0)                                  AS quantity,
  CASE
    WHEN COALESCE(price_per_unit,0) <> 0
     AND COALESCE(quantity,0)      <> 0
     AND (total_spent IS NULL OR ROUND(total_spent,2) <> ROUND(price_per_unit*quantity,2))
    THEN ROUND(price_per_unit*quantity, 2)
    ELSE COALESCE(total_spent, ROUND(COALESCE(price_per_unit,0)*COALESCE(quantity,0),2))
  END                                                    AS total_spent,
  COALESCE(payment_method, 'unknown')                    AS payment_method,
  COALESCE(location,       'unknown')                    AS location,
  /* Keep unknown dates as the literal 'Unknown' (TEXT) */
  CASE
    WHEN transaction_date IS NULL THEN 'Unknown'
    ELSE CAST(transaction_date AS TEXT)
  END                                                    AS transaction_date,
  COALESCE(discount_applied, 0)                          AS discount_applied
FROM cleaned_dedup;
""")

q("SELECT * FROM cleaned_final LIMIT 5;")

,transaction_id,customer_id,category,item,price_per_unit,quantity,total_spent,payment_method,location,transaction_date,discount_applied
0,TXN_1002182,CUST_01,Food,Item_5_FOOD,11.0,5,55.0,digital wallet,in store,2024-10-08,1
1,TXN_1003865,CUST_15,Furniture,Item_2_FUR,6.5,5,32.5,cash,online,2022-03-12,0
2,TXN_1003940,CUST_06,Furniture,Item_5_FUR,11.0,9,99.0,digital wallet,online,2022-04-22,0
3,TXN_1004091,CUST_04,Food,Item_25_FOOD,41.0,3,123.0,cash,in store,2023-11-09,0
4,TXN_1004124,CUST_08,Computers and electric accessories,Item_7_CEA,14.0,5,70.0,credit card,in store,2022-03-02,0


In [20]:
kpis = {}

kpis["rows_raw"]       = q("SELECT COUNT(*) c FROM raw_data;")["c"][0]
kpis["rows_stage"]     = q("SELECT COUNT(*) c FROM stage;")["c"][0]
kpis["rows_dedup"]     = q("SELECT COUNT(*) c FROM cleaned_dedup;")["c"][0]
kpis["rows_final"]     = q("SELECT COUNT(*) c FROM cleaned_final;")["c"][0]
kpis["dups_removed"]   = kpis["rows_stage"] - kpis["rows_dedup"]

kpis["nulls_raw"] = q("""
SELECT
  SUM("Transaction ID" IS NULL) +
  SUM("Customer ID"    IS NULL) +
  SUM(Category         IS NULL) +
  SUM(Item             IS NULL) +
  SUM("Price Per Unit" IS NULL) +
  SUM(Quantity         IS NULL) +
  SUM("Total Spent"    IS NULL) +
  SUM("Payment Method" IS NULL) +
  SUM(Location         IS NULL) +
  SUM("Transaction Date" IS NULL) +
  SUM("Discount Applied" IS NULL) AS total_nulls
FROM raw_data;""")["total_nulls"][0]

# In final table, we imputed everything (including date -> 'Unknown'), so nulls should be 0
kpis["nulls_final"] = q("""
SELECT
  SUM(transaction_id   IS NULL) +
  SUM(customer_id      IS NULL) +
  SUM(category         IS NULL) +
  SUM(item             IS NULL) +
  SUM(price_per_unit   IS NULL) +
  SUM(quantity         IS NULL) +
  SUM(total_spent      IS NULL) +
  SUM(payment_method   IS NULL) +
  SUM(location         IS NULL) +
  SUM(transaction_date IS NULL) +
  SUM(discount_applied IS NULL) AS total_nulls
FROM cleaned_final;""")["total_nulls"][0]

# Invalid dates in raw (non-parseable)
kpis["bad_dates_raw"] = q("""
SELECT COUNT(*) c FROM raw_data
WHERE "Transaction Date" IS NOT NULL AND julianday("Transaction Date") IS NULL;""")["c"][0]

# In final, no invalid dates because 'Unknown' is literal text; check remaining real-date strings:
kpis["real_dates_final"] = q("""
SELECT COUNT(*) c FROM cleaned_final
WHERE transaction_date != 'Unknown' AND julianday(transaction_date) IS NOT NULL;""")["c"][0]

# Totals mismatches after fix (should be 0)
kpis["total_mismatches_final"] = q("""
SELECT COUNT(*) c FROM cleaned_final
WHERE total_spent IS NOT NULL
  AND price_per_unit IS NOT NULL
  AND quantity IS NOT NULL
  AND ROUND(total_spent,2) <> ROUND(price_per_unit*quantity,2);
""")["c"][0]

kpis

{'rows_raw': np.int64(12575),
 'rows_stage': np.int64(12575),
 'rows_dedup': np.int64(12575),
 'rows_final': np.int64(12575),
 'dups_removed': np.int64(0),
 'nulls_raw': np.int64(7229),
 'nulls_final': np.int64(0),
 'bad_dates_raw': np.int64(0),
 'real_dates_final': np.int64(12575),
 'total_mismatches_final': np.int64(609)}

In [21]:
# BEFORE (raw): order by date desc; unknown/invalid last
before = q("""
SELECT * FROM raw_data
ORDER BY
  CASE WHEN julianday("Transaction Date") IS NULL THEN 1 ELSE 0 END,
  julianday("Transaction Date") DESC
LIMIT 10;
""")
print("Before (most recent first, unknown/invalid last):")
display(before)

# AFTER (final): order by date desc; 'Unknown' last
after = q("""
SELECT * FROM cleaned_final
ORDER BY
  CASE WHEN transaction_date = 'Unknown' THEN 1 ELSE 0 END,
  julianday(transaction_date) DESC;
""")
print("After (most recent first, 'Unknown' last):")
display(after.head(10))

Before (most recent first, unknown/invalid last):


,Transaction ID,Customer ID,Category,Item,Price Per Unit,Quantity,Total Spent,Payment Method,Location,Transaction Date,Discount Applied
0,TXN_7369318,CUST_01,Beverages,Item_11_BEV,20.0,5.0,100.0,Credit Card,Online,2025-01-18,0.0
1,TXN_1777313,CUST_07,Milk Products,None,15.5,NaN,NaN,Credit Card,Online,2025-01-18,0.0
2,TXN_5321918,CUST_24,Furniture,Item_6_FUR,12.5,5.0,62.5,Credit Card,In-store,2025-01-18,0.0
3,TXN_6383632,CUST_12,Electric household essentials,Item_21_EHE,35.0,2.0,70.0,Cash,In-store,2025-01-18,NaN
4,TXN_1505827,CUST_13,Computers and electric accessories,None,NaN,3.0,109.5,Digital Wallet,Online,2025-01-18,NaN
5,TXN_5907338,CUST_25,Beverages,Item_23_BEV,38.0,9.0,342.0,Cash,In-store,2025-01-18,1.0
6,TXN_5804265,CUST_11,Computers and electric accessories,Item_20_CEA,33.5,2.0,67.0,Digital Wallet,Online,2025-01-18,1.0
7,TXN_1661883,CUST_04,Beverages,Item_17_BEV,29.0,9.0,261.0,Cash,In-store,2025-01-18,0.0
8,TXN_5709336,CUST_10,Food,Item_4_FOOD,9.5,2.0,19.0,Credit Card,In-store,2025-01-18,1.0
9,TXN_2596812,CUST_25,Patisserie,Item_2_PAT,6.5,8.0,52.0,Cash,Online,2025-01-17,NaN


After (most recent first, 'Unknown' last):


,transaction_id,customer_id,category,item,price_per_unit,quantity,total_spent,payment_method,location,transaction_date,discount_applied
0,TXN_1505827,CUST_13,Computers and electric accessories,unknown,0.0,3,109.5,digital wallet,online,2025-01-18,0
1,TXN_1661883,CUST_04,Beverages,Item_17_BEV,29.0,9,261.0,cash,in store,2025-01-18,0
2,TXN_1777313,CUST_07,Milk Products,unknown,15.5,0,0.0,credit card,online,2025-01-18,0
3,TXN_5321918,CUST_24,Furniture,Item_6_FUR,12.5,5,62.5,credit card,in store,2025-01-18,0
4,TXN_5709336,CUST_10,Food,Item_4_FOOD,9.5,2,19.0,credit card,in store,2025-01-18,1
5,TXN_5804265,CUST_11,Computers and electric accessories,Item_20_CEA,33.5,2,67.0,digital wallet,online,2025-01-18,1
6,TXN_5907338,CUST_25,Beverages,Item_23_BEV,38.0,9,342.0,cash,in store,2025-01-18,1
7,TXN_6383632,CUST_12,Electric household essentials,Item_21_EHE,35.0,2,70.0,cash,in store,2025-01-18,0
8,TXN_7369318,CUST_01,Beverages,Item_11_BEV,20.0,5,100.0,credit card,online,2025-01-18,0
9,TXN_1011669,CUST_14,Food,Item_13_FOOD,23.0,8,184.0,credit card,online,2025-01-17,0


In [22]:
q("SELECT * FROM cleaned_final;").to_csv("DirtyRetailSales_cleaned.csv", index=False)
print("Saved: DirtyRetailSales_cleaned.csv")

Saved: DirtyRetailSales_cleaned.csv
